In [4]:
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: sample_submission.csv   
  inflating: data_description.txt    
  inflating: test.csv                
  inflating: train.csv               


In [0]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns

In [0]:
import pandas as pd

'''
データのロード
'''
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [0]:
pd.set_option('display.max_columns', 90)

In [148]:
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [183]:
# trainとtestのラベルを追加
train['Type'] = 1
test['Type'] = 0
data = train.append(test)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [150]:
'''
データのクリーニング
'''

'\nデータのクリーニング\n'

In [0]:
def missing_table(df):      # データの欠損を計算
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    missing_table = pd.concat([null_val, percent], axis=1)
    missing_table_ren_columns = missing_table.rename(
        columns = {0:"欠損数", 1:"%"}
    )
    return missing_table_ren_columns

In [0]:
def learge_missing(df, rate):
  index = []
  name = []
  null_val = df.isnull().sum()
  percent = 100 * df.isnull().sum()/len(df)
  for i in range(len(percent)):
    if percent[i] > rate:
      index.append(i)
  _ = null_val[index]
  name.append(_.index)
#   print(100 * data[list(np.array(name).flatten())].isnull().sum()/len(data[list(np.array(name).flatten())]))
  return list(np.array(name).flatten())

In [0]:
# Nanをラベルにしているコラムの調整
titles = learge_missing(data, 30)[:-1]
for title in titles:
  data[title].fillna('Else', inplace = True)

In [0]:
# 欠損値の補完
miss_title = [s for s in learge_missing(data, 0) if 'SalePrice' not in s]
for title in miss_title:
  data[title].fillna(data[title].mode()[0], inplace = True)

In [0]:
# コラムのエンコード

target_col = ["SalePrice"]
id_dataset = ["Type"]
cat_cols   = data.nunique()[data.nunique() < 6].keys().tolist()   # unique数が26未満のラベルをcat_colsにまとめる
cat_cols   = [x for x in cat_cols ]
# numerical columns(数値)
num_cols   = [x for x in data.columns if x not in cat_cols + target_col + id_dataset]  # 数値ラベルをまとめる
# Columns more than 1 values(2種以上のクラス)
multi_cols = [i for i in cat_cols if i not in id_dataset]

unique数を幾つに設定するかは要検証

In [0]:
# multi_colsをエンコード
le = LabelEncoder()
for i in multi_cols :
    data[i] = le.fit_transform(data[i])

In [0]:
damy = data.copy()

In [0]:
# 多ラベルのコラムを、ラベルごとのSalePrice平均値に変換
# _ = data
# _.dropna()

titles = [s for s in multi_cols if s not in id_dataset]
for title in titles:
  unique_list = list(data[title].unique())
  for unique in unique_list:
    data.loc[data[title]==unique, title] = data[data[title] == unique]['SalePrice'].mean()

In [0]:
# 例外的なデータ(全のデータ中に一種しかない)の処理
# data['3SsnPorch'].fillna(damy[damy['3SsnPorch'] != 0]['SalePrice'].mean(), inplace=True)
data['Fireplaces'].fillna(damy[damy['Fireplaces'] == 3]['SalePrice'].quantile(.75), inplace=True)
data['FullBath'].fillna(damy[damy['FullBath'] == 3]['SalePrice'].quantile(.75), inplace=True)
# data['GarageCars'].fillna(damy[damy['GarageCars'] == 4]['SalePrice'].quantile(.75), inplace=True)
# data['LowQualFinSF'].fillna(damy[damy['LowQualFinSF'] != 0]['SalePrice'].mean(), inplace=True)
# data['MSSubClass'].fillna(damy[damy['MSSubClass'] == 4]['SalePrice'].quantile(.25), inplace=True)
# data['MiscVal'].fillna(damy[damy['MiscVal'] != 0]['SalePrice'].mean(), inplace=True)
# data['PoolArea'].fillna(damy[damy['PoolArea'] != 0]['SalePrice'].mean(), inplace=True)
# data['TotRmsAbvGrd'].fillna(damy[damy['TotRmsAbvGrd'] == 10]['SalePrice'].quantile(.75), inplace=True)

In [0]:
# for i in range(damy['TotRmsAbvGrd'].nunique()):
#   print(i, damy[damy['TotRmsAbvGrd'] == i]['SalePrice'].mean())

In [0]:
# data['TotRmsAbvGrd'].value_counts()

In [0]:
# damy['TotRmsAbvGrd'].value_counts()

In [193]:
learge_missing(data, 0)

['Fireplaces', 'FullBath', 'SalePrice']

In [0]:
cols = data.columns.tolist()   # ラベルを格納
cols.insert(0, cols.pop(cols.index('SalePrice')))
data = data.reindex(columns= cols)   # Survivedを先頭にする

In [0]:
# dataの分離
train = data[data['Type'] == 1].drop(columns = ['Type', 'Id'])
test = data[data['Type'] == 0].drop(columns = ['Type', 'Id'])

In [0]:
drop_cols = train.select_dtypes(include=object).columns.tolist()
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

In [0]:
all_cols = train.columns.tolist()

In [200]:
# num_colsを標準化
std = StandardScaler()
scaled = std.fit(train[all_cols])
scaled_train = std.transform(train[all_cols])
scaled_test = std.transform(test[all_cols])
scaled_train = pd.DataFrame(scaled_train,columns = all_cols)
scaled_test = pd.DataFrame(scaled_test,columns = all_cols)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [201]:
train.head()

,SalePrice,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,208500.0,856,854,0,183452.131483,3,185763.807377,181492.227745,163439.248234,706.0,0.0,2,5,202522.918367,181230.330189,202688.478964,150.0,186186.70989,2,2,186810.637453,0,184034.896256,231633.510246,187596.837998,0,141331.482609,2,213009.825521,6,548.0,2,183017.189055,202068.869668,182591.864224,1,2003.0,1710,212721.960748,1,214914.429150,5,183388.79023,212116.023891,180183.746758,179956.799566,8450,176938.047529,65.0,164754.818378,0,60,191004.994787,196.0,204691.871910,182046.410384,0,2,61,4,7,186433.973881,0,180404.663455,1,1,4,8,0,181130.538514,8,856.0,180950.95682,0,2003,2003,177360.838816
1,181500.0,1262,0,0,183452.131483,3,185763.807377,181492.227745,257689.805970,978.0,0.0,0,5,165521.640187,176098.125000,202688.478964,284.0,186186.70989,1,2,186810.637453,0,184034.896256,144341.313466,187596.837998,5,211843.909231,1,213009.825521,6,460.0,2,183017.189055,202068.869668,182591.864224,1,1976.0,1262,162534.884995,1,214914.429150,2,183388.79023,139962.511565,180183.746758,179956.799566,9600,177934.574468,80.0,164754.818378,0,20,191004.994787,0.0,156958.243119,182046.410384,0,5,0,7,6,186433.973881,0,180404.663455,1,1,4,8,0,181130.538514,6,1262.0,180950.95682,298,1976,1976,186063.151976
2,223500.0,920,866,0,183452.131483,3,185763.807377,181492.227745,192789.657895,486.0,0.0,2,5,202522.918367,181230.330189,202688.478964,434.0,186186.70989,2,2,186810.637453,0,184034.896256,231633.510246,187596.837998,5,211843.909231,2,213009.825521,6,608.0,2,183017.189055,202068.869668,182591.864224,1,2001.0,1786,212721.960748,1,214914.429150,5,183388.79023,212116.023891,180183.746758,179956.799566,11250,176938.047529,68.0,206101.665289,0,60,191004.994787,162.0,204691.871910,182046.410384,0,9,42,4,7,186433.973881,0,180404.663455,1,1,4,8,0,181130.538514,6,920.0,180950.95682,0,2001,2002,177360.838816
3,140000.0,961,756,0,183452.131483,3,185763.807377,213599.907692,163439.248234,216.0,0.0,0,5,202522.918367,181230.330189,138866.295918,540.0,186186.70989,2,2,186810.637453,272,184034.896256,144341.313466,187596.837998,3,211843.909231,0,134751.440000,6,642.0,3,183017.189055,137570.460641,182591.864224,5,1998.0,1717,162534.884995,1,156858.871369,5,183388.79023,212116.023891,180183.746758,179956.799566,9550,181623.425856,60.0,206101.665289,0,70,191004.994787,0.0,156958.243119,182046.410384,0,2,35,4,7,186433.973881,0,180404.663455,1,1,0,8,0,181130.538514,7,756.0,180950.95682,0,1915,1970,182549.458599
4,250000.0,1145,1053,0,183452.131483,4,185763.807377,181492.227745,206643.420814,655.0,0.0,2,5,202522.918367,181230.330189,202688.478964,490.0,186186.70989,2,2,186810.637453,0,184034.896256,231633.510246,187596.837998,5,211843.909231,2,213009.825521,6,836.0,3,183017.189055,202068.869668,182591.864224,1,2000.0,2198,212721.960748,1,214914.429150,5,183388.79023,212116.023891,180183.746758,179956.799566,14260,177934.574468,84.0,206101.665289,0,60,191004.994787,350.0,204691.871910,182046.410384,0,12,84,4,8,186433.973881,0,180404.663455,1,1,4,8,0,181130.538514,9,1145.0,180950.95682,192,2000,2000,177360.838816


In [0]:
data['SaleType'].nunique()

In [0]:
# 不要なラベルの削除とscaledのmerge
train = train.drop(columns = all_cols,axis = 1)
train = train.merge(scaled_train,left_index = True,right_index = True,how = "left")

test = test.drop(columns = all_cols,axis = 1)
test = test.merge(scaled_test,left_index = True,right_index = True,how = "left")

In [203]:
# trainを入力と正解に分離しarrayに変換
X_train = train.iloc[:, 1:].as_matrix()
Y_train = train.iloc[:,0].as_matrix()
X_test = test.iloc[:, 1:].as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
'''
データの保存
'''
import pickle

pickle_file = "./tuned_data5.pickle"
with open(pickle_file, 'wb') as f:
    pickle.dump(X_train, f)
    pickle.dump(Y_train, f)
    pickle.dump(X_test, f)

In [0]:
with open(pickle_file, 'rb') as f:
    X_train = pickle.load(f)
    Y_train = pickle.load(f)
    X_test = pickle.load(f)

In [207]:
X_test.shape

(1459, 76)